In [1]:
library(Seurat)
library(tidyverse)
library(patchwork)
library(targets)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
meta = tar_meta(store = '../01_parameter_sweep/_targets/')

In [9]:
meta %>%
filter(str_detect(name, "batchy_score_tibble_Day14.Astrocytes_1bf64242363c591aa0e36f5b82d04c8e")) %>%
filter(!is.na(error))

name,type,data,command,depend,seed,path,time,size,bytes,format,repository,iteration,parent,children,seconds,warnings,error
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<list>,<dttm>,<chr>,<int64>,<chr>,<chr>,<chr>,<lgl>,<list>,<dbl>,<chr>,<chr>
batchy_score_tibble_Day14.Astrocytes_1bf64242363c591aa0e36f5b82d04c8e,stem,error,272a2dc45bfaf186,0840cff6286d9ba2,1368625049,NA,2024-01-14 21:24:10,de1c6d017cb4e8a8,38,qs,local,vector,NA,NA,0.004,NA,unused argument .


In [10]:
qs::qread('../01_parameter_sweep/_targets/objects/batchy_score_tibble_Day5.Astrocytes_ae00ffb5b995922e7a1ce470f5d7fd28')

name,batchy_score
<chr>,<dbl>
Day5.Astrocytes_ae00ffb5b995922e7a1ce470f5d7fd28,14723.27


In [ ]:
meta %>%
filter(str_detect(name, "Astrocytes")) %>%
filter(is.na(error))

In [ ]:
# obj = qs::qread('../01_milo/_targets/objects/obj_Day14.Agrp')
obj = qs::qread('../01_parameter_sweep/_targets/objects/obj_Day5.Astrocytes_04840fcc9a6fcd161d85d9ccc1c994d4')
obj

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
DimPlot(obj, reduction = "umap", group.by='batch', label=T) + NoLegend()

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
DimPlot(obj, reduction = "umap", group.by='seurat_clusters', label=T) + NoLegend()

In [ ]:
# obj_other = qs::qread('../00_cellbender/_targets/objects/obj_cb_other_01')
obj_other = qs::qread('_targets/objects/obj_other')
obj_other

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
DimPlot(obj_other, reduction = "umap", group.by='labels_lvl1', label=F) + NoLegend()

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
DimPlot(obj_other, reduction = "umap", group.by='seurat_clusters', label=F) + NoLegend()

In [ ]:
options(repr.plot.width=18, repr.plot.height=6)
DimPlot(obj, reduction = "umap", group.by='batch', split.by='batch') + NoLegend()

In [ ]:
obj = obj %>% FindNeighbors(reduction = 'pca', 
                            dims = 1:30, 
                            return.neighbor = TRUE, 
                            k.param = 40, 
                            graph.name = c('nn_graph'))

In [ ]:
obj = obj %>% FindNeighbors(reduction = 'umap', 
                            dims = 1:2, 
                            return.neighbor = TRUE, 
                            k.param = 40, 
                            graph.name = c('nn_graph'))

In [ ]:
obj@neighbors

In [ ]:
Cells(obj) %>% sample(1)

In [ ]:
sampled_cell = Cells(obj) %>% sample(1)
top_neighbors_cell = TopNeighbors(obj@neighbors$nn_graph, sampled_cell, 40)

options(repr.plot.width=12, repr.plot.height=6)
p_cell = DimPlot(obj, cells.highlight = sampled_cell) + NoLegend()
p_neighbors = DimPlot(obj, cells.highlight = top_neighbors_cell) + NoLegend()
p_cell + p_neighbors

In [ ]:
chi_squared_test <- function(batches, all_batches) {
  # Calculate the table of observed frequencies for batches
  observed <- table(batches)

  # Calculate the table of frequencies for all_batches
  all_batches_freq <- table(all_batches)

  # Calculate the expected frequencies
  expected <- all_batches_freq * length(batches) / length(all_batches)

  # Match the names in observed and expected
  # Assign 0 to categories in expected that are not in observed
  expected <- expected[names(all_batches_freq)]
  names(expected) <- names(all_batches_freq)
  observed <- observed[names(all_batches_freq)]
  observed[is.na(observed)] <- 0

  # Perform the Chi-Squared test
  test_result <- chisq.test(x = observed, p = expected / sum(expected))

  # Return the test result
  return(test_result)
}


In [ ]:
all_batches = obj$batch

df = Cells(obj) %>% 
#     head(11) %>%
    tibble(barcode=.) %>%
    rowwise %>%
    mutate(top_neighbors = list(TopNeighbors(obj@neighbors$nn5, barcode, 25))) %>%
    mutate(batches = obj %>% 
                   `[[` %>%
                   filter(row.names(.) %in% unlist(top_neighbors)) %>%
                   pull(batch) %>%
                   list) %>%
    mutate(batch_xsqp = chi_squared_test(batches %>% unlist, all_batches) %>%
                        `$`('p.value')) %>%
    ungroup

In [ ]:
df_meta = df %>%
select(barcode, batch_xsqp) %>%
column_to_rownames(var = 'barcode')

obj = obj %>% AddMetaData(df_meta)

In [ ]:
batchy_score = function(obj=obj,
                        batch = batch,
                        dims = dims,
                        reduction = 'pca',
                        k.param = 40){

    obj = obj %>% FindNeighbors(reduction = reduction, 
                            dims = seq(dims), 
                            return.neighbor = TRUE, 
                            k.param = k.param, 
                            graph.name = c('nn_graph'))
    all_batches = obj[[batch]]

    df = Cells(obj) %>% 
        tibble(barcode=.) %>%
        rowwise %>%
        mutate(top_neighbors = list(TopNeighbors(obj@neighbors$nn_graph, barcode, k.param))) %>%
        mutate(batches = obj %>% 
                       `[[` %>%
                       filter(row.names(.) %in% unlist(top_neighbors)) %>%
                       `[[`(batch) %>%
                       list) %>%
        mutate(batch_xsqp = chi_squared_test(batches %>% unlist, all_batches) %>%
                            `$`('p.value')) %>%
        ungroup
    
    df_meta = df %>%
        select(barcode, batch_xsqp) %>%
        column_to_rownames(var = 'barcode') %>%
        rowwise %>%
        mutate(logp_bxsqp = -log10(batch_xsqp)) %>%
        ungroup

    obj = obj %>% AddMetaData(df_meta)
    obj
}

In [ ]:
obj = batchy_score(obj=obj,
                        batch = "batch",
                        dims = 30,
                        reduction = 'pca',
                        k.param = 40)

obj

In [ ]:
obj %>% `[[`('logp_bxsqp') %>% sum %>% tibble(name='blah', batchy_score=.)

In [ ]:
df_meta %>%
rowwise %>%
mutate(logp_bxsqp = -log10(batch_xsqp)) %>%
ungroup

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
FeaturePlot(obj, reduction = "umap", features = 'batch_xsqp', min.cutoff = 0.05, max.cutoff=0.050001)

In [ ]:
obj %>% 
`[[` %>%
mutate(batch_here = batch_xsqp < 0.05) %>%
group_by(batch, batch_here) %>%
summarise(n=n()) %>% print

In [ ]:
obj$batch_here = obj %>% 
`[[` %>%
mutate(batch_here = batch_xsqp < 0.05)

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
DimPlot(obj, reduction = "umap", group.by='batch_here', label=F)

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)
DimPlot(obj, reduction = "umap", split.by='batch_here', group.by='batch_here', label=F)